[![Open Rendered Output](https://img.shields.io/badge/Rendered%20Output-Open-blue?logo=link&logoColor=white)](https://htmlpreview.github.io/?https://raw.githubusercontent.com/MeteoSwiss/nwp-fdb-polytope-demo/main/examples/snapshots/feature_trajectory.html)

# Trajectory Feature: Trajectories between MeteoSwiss Stations 

The following notebook demonstrates the full workflow to access, process and visualize trajectories of ICON-CH1-EPS temperature data between four MeteoSwiss locations. 

<div style="text-align:center;">
  <img src="https://raw.githubusercontent.com/MeteoSwiss/nwp-fdb-polytope-demo/main/notebooks/Polytope/images/t_2m_trajectory.png" style="width:50%;"/>
</div>

The data is retrieved using [Polytope](https://polytope.readthedocs.io/en/latest/), a feature extraction software developed by ECMWF. It applies concepts of computational geometry to extract n-dimensional polygons (also known as polytopes) from datacubes, such as trajectories. To access MeteoSwiss' operational ICON-CH1-EPS and ICON-CH2-EPS model data, [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) provides a wrapper around the polytope client that simplifies the request API. Follow the instructions to learn more about model data access via Polytope.

## Installation
Follow the instructions in [README.md](https://github.com/MeteoSwiss/nwp-fdb-polytope-demo/blob/main/README.md#Installation-1) to install the necessary dependencies.

## Configuring Access to Polytope
To access ICON data via MeteoSwiss's Polytope, you need a Polytope offline token provided by MeteoSwiss. If you do not already have a token, you can request one [here](https://meteoswiss.atlassian.net/wiki/spaces/IW2/pages/327780397/Polytope#Offline-token-authentication). Then, create a new `config.yml` file based on [`config_example.yml`](config_example.yml), and replace <meteoswiss_key> with your access token there. 

In [ ]:
import os
import yaml

def load_config(path="config.yml"):
    if not os.path.exists(path):
        raise FileNotFoundError("Missing config.yml. Please create one based on config_example.yml.")
    with open(path, "r") as f:
        return yaml.safe_load(f)

config = load_config()

#ICON-CSCS Polytope credentials
os.environ["POLYTOPE_USER_KEY"] = config["meteoswiss"]["key"]
os.environ["POLYTOPE_ADDRESS"] = "https://polytope-depl.mchml.cscs.ch"

## Selecting date and time of the forecast

The realtime FDB typically **includes only the most recent day of forecasts**. Therefore, it is necessary to specify the current date and select a corresponding forecast time in the past.

In [ ]:
from datetime import datetime, timedelta

# Current time
now = datetime.now()

# Subtract 12 hours
past_time = now - timedelta(hours=12)

# Round down to the nearest multiple of 6
rounded_hour = (past_time.hour // 6) * 6
rounded_time = past_time.replace(hour=rounded_hour, minute=0, second=0, microsecond=0)

# Format as YYYYMMDD and HHMM
date = rounded_time.strftime('%Y%m%d')
time = rounded_time.strftime('%H%M')
date,time

## Select the geolocation
We’ll request trajectories connecting all MeteoSwiss stations.

To use the Polytope feature for extracting trajectories, it is necessary to rotate the given data points (here Zurich Airport, Payerne, Geneva and Locarno) since the data source accessed by Polytope is stored on a rotated grid. By using a South Pole rotation with a reference of longitude 10° and latitude of -43°, we are able to access the desired data points. 
> **IMPORTANT**: The function `transform_point()` expects first longitude and then latitude.

In [ ]:
import cartopy.crs as ccrs

# points for the trajectory
geo_points_dict = [
    {"name" : "Zurich Airport", "longitude" : 8.565, "latitude" : 47.454},
    {"name" : "Payerne", "longitude" : 6.933, "latitude" : 46.817},
    {"name" : "Geneva", "longitude" : 6.143, "latitude" : 46.204},
    {"name" : "Locarno", "longitude" : 8.799, "latitude" : 46.170},
]

geo_points = [(city["longitude"], city["latitude"]) for city in geo_points_dict]

# South pole rotation of lon=10, latitude=-43
rotated_crs = ccrs.RotatedPole(
    pole_longitude=190, pole_latitude=43
)

# Convert a point from geographic to rotated coordinates
geo_crs = ccrs.PlateCarree()
rotated_points = [
    rotated_crs.transform_point(lon, lat, geo_crs)
    for lon, lat in geo_points
]

## Define the request

Once the points are rotated, we need to define a mars request using [meteodata-lab](https://polytope.readthedocs.io/en/latest/). The `feature` attribute allows you to extract **only the relevant data at the given points**. Thus, the amount of data that is retrieved from storage is significantly reduced. For the "trajectory" `feature` the following dictionary is needed.


In [ ]:
feature={
    "type" : "trajectory",
    "points" : rotated_points,
    "inflation" : 0.05,
    "inflate" : 'round',
    "axes" : ["longitude", "latitude"],
}

- The `inflation` key defines the size of the steps along the trajectory. If it is a single value, this will be the inflation along `longitude` and `latitude`. If it is a list of values, each value will correspond to the inflation of the corresponding axis described in the `axes` key. <br>
- The `inflate` key defines the shape that flyes along the trajectory. For a 2D-field you can choose between `round` and `box`.

Finally, we can define the request. This example fetches **2-meter temperature** from **ICON-CH1-EPS** at the **surface**, for the **control forecast (cf)**, at the selected run date/time.

In [ ]:
from meteodatalab import mars

request = mars.Request(
    param="T_2M",
    date=date,
    time=time,
    model=mars.Model.ICON_CH1_EPS,
    levtype=mars.LevType.SURFACE,
    type="cf",
    step=0,
    feature=feature
)

## Data retrieval
Now we use [earthkit.data](https://earthkit-data.readthedocs.io/en/latest/) to load the data and convert it into an [xarray.Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html).

In [ ]:
import earthkit.data as ekd
ds = ekd.from_source(
    "polytope",
    "mchgj",
    request.to_polytope(),
    stream=False
).to_xarray()

## Reverse rotation
The data in the dataset contains rotated longitudes and latitudes. To plot it, we will reverse the rotation.

In [ ]:
unrotated_points = [
    geo_crs.transform_point(lon, lat, rotated_crs)
    for lon, lat in zip(ds.longitude,ds.latitude)
]

geo_lons, geo_lats = zip(*unrotated_points)

ds_geo = ds.assign_coords(
    longitude=("points", list(geo_lons)),
    latitude=("points", list(geo_lats))
)

## Plotting

We use the library [earthkit.plots](https://earthkit-plots.readthedocs.io/en/latest/) to plot the data. Moreover, we can specify a styling with [earthkit.plots.styles](https://earthkit-plots.readthedocs.io/en/latest/examples/guide/04-styles.html#4.-Styles) to show a color spectrum from red to blue and convert the unit to degrees Celsius for better readability.

In [ ]:
from earthkit.plots import Map
from earthkit.plots.styles import Style
from earthkit.plots.geo import bounds, domains

xmin, xmax = 5, 11   # Longitude bounds
ymin, ymax = 45.5, 48   # Latitude bounds

crs = ccrs.Geodetic()

bbox = bounds.BoundingBox(xmin, xmax, ymin, ymax, crs)
domain = domains.Domain.from_bbox(
    bbox=bbox,
    name="CH2"
)

# plot trajectories
style = Style(colors="RdBu_r", units="celsius")
chart = Map(domain=domain)
chart.point_cloud(ds_geo["t_2m"], x="longitude", y="latitude", style=style)

# plot MeteoSwiss' stations
lons = [city["longitude"]for city in geo_points_dict]
lats = [city["latitude"] for city in geo_points_dict]
names = [city["name"] for city in geo_points_dict]

ax = chart.ax
ax.plot(lons, lats, ".", color="black", markersize=10, transform=crs)

# add features
chart.coastlines()
chart.borders()
chart.gridlines()

# add title and legend
chart.title("Trajectory Feature: 2-Meter Temperature at {datetimes}")
chart.legend()

# labels for MeteoSwiss' stations
for lon, lat, name in zip(lons, lats, names):
    ax.text(
        lon, lat+0.1, name,
        transform=crs,
        fontsize=12,
        color="black",
        ha="center",
        va="bottom"
    )

chart.show()